# Problem statement

Sudden user engagement dip in August requires investigation.

# Possible causes of retention drop

1. 
2. 
3. 


# Digging into the data

Importing all necessary packages

In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

Connecting to the database

In [2]:
conn = sqlite3.connect('case-study.db')
c = conn.cursor()

Exploring users data table

In [3]:
pd.read_sql('''
SELECT * FROM users
''', conn).head()

,user_id,created_at,company_id,language,activated_at,state
0,0,2013-01-01 20:59:39,5737,english,2013-01-01 21:01:07,active
1,1,2013-01-01 13:07:46,28,english,None,pending
2,2,2013-01-01 10:59:05,51,english,None,pending
3,3,2013-01-01 18:40:36,2800,german,2013-01-01 18:42:02,active
4,4,2013-01-01 14:37:51,5110,indian,2013-01-01 14:39:05,active
